Step 1: Collect raw data.

In [24]:
import urllib.request
import datetime
import os

In [25]:
for i in range(1,28):
    url="https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean".format(i)
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.datetime.now()
    date_and_time_time = now.strftime("%Y%m%d%H%M%S")
    out = open(os.path.join('raw', 'NOAA_ID'+str(i)+'_'+date_and_time_time+'.csv'), 'wb')
    out.write(text)
    out.close()

Step 2: Creat dataframe.

In [3]:
import pandas as pd

In [13]:
directory = 'raw'
final_list = []
for i in range(1,28):
    new_list = [x for x in os.listdir(directory) if x.split('_')[1] == f'ID{i}']
    latest_file = sorted(new_list, key=lambda x: int(x.split('_')[2][:-4]))[-1]
    final_list.append(latest_file)




print(final_list)
headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'Area']


index_changer = {
    1:'Черкаська', 2:'Чернігівська', 3:'Чернівецька', 4:'Республіка Крим', 5:'Дніпропетровська',
    6:'Донецька', 7:'Івано-Франківська', 8:'Харківська', 9:'Херсонська', 10:'Хмельницька',
    11:'Київська', 12:'Київ', 13:'Кіровоградська', 14:'Луганська', 15:'Львівська',
    16:'Миколаївська', 17:'Одеська', 18:'Полтавська', 19:'Рівеньска', 20:'Севастопіль',
    21:'Сумьска', 22:'Тернопільська', 23:'Закарпатська', 24:'Вінницька', 25:'Волинська',
    26:'Запорізька', 27:'Житомирська',
}

['NOAA_ID1_20240311182803.csv', 'NOAA_ID2_20240311182805.csv', 'NOAA_ID3_20240311182807.csv', 'NOAA_ID4_20240311182808.csv', 'NOAA_ID5_20240311182809.csv', 'NOAA_ID6_20240311182811.csv', 'NOAA_ID7_20240311182812.csv', 'NOAA_ID8_20240311182814.csv', 'NOAA_ID9_20240311182816.csv', 'NOAA_ID10_20240311182817.csv', 'NOAA_ID11_20240311182818.csv', 'NOAA_ID12_20240311182819.csv', 'NOAA_ID13_20240311182820.csv', 'NOAA_ID14_20240311182821.csv', 'NOAA_ID15_20240311182823.csv', 'NOAA_ID16_20240311182825.csv', 'NOAA_ID17_20240311182826.csv', 'NOAA_ID18_20240311182827.csv', 'NOAA_ID19_20240311182828.csv', 'NOAA_ID20_20240311182829.csv', 'NOAA_ID21_20240311182831.csv', 'NOAA_ID22_20240311182832.csv', 'NOAA_ID23_20240311182833.csv', 'NOAA_ID24_20240311182835.csv', 'NOAA_ID25_20240311182837.csv', 'NOAA_ID26_20240311182839.csv', 'NOAA_ID27_20240311182840.csv']


In [16]:
i = 1
uncombined_df = []
for file in final_list:
    file_path = os.path.join(directory, file)
    dataframe = pd.read_csv(file_path, header = 1, names = headers)
    dataframe.drop(dataframe.index[-1], inplace=True)
    dataframe.iloc[0, 0] = dataframe.iloc[0, 0].replace('<tt><pre>', '')
    dataframe['Area'] = i
    i += 1

    dataframe = dataframe.drop(dataframe.loc[dataframe['VHI'] == -1].index)
    dataframe['Area'] = dataframe['Area'].replace({index: area for index, area in index_changer.items()})

    uncombined_df.append(dataframe)

df = pd.concat(uncombined_df, ignore_index=True)
print(df)

       Year  Week    SMN     SMT    VCI    TCI    VHI         Area
0      1982   1.0  0.053  260.31  45.01  39.46  42.23    Черкаська
1      1982   2.0  0.054  262.29  46.83  31.75  39.29    Черкаська
2      1982   3.0  0.055  263.82  48.13  27.24  37.68    Черкаська
3      1982   4.0  0.053  265.33  46.09  23.91  35.00    Черкаська
4      1982   5.0  0.050  265.66  41.46  26.65  34.06    Черкаська
...     ...   ...    ...     ...    ...    ...    ...          ...
57883  2024   6.0  0.099  265.49  53.55  40.85  47.20  Житомирська
57884  2024   7.0  0.114  267.82  57.51  37.63  47.57  Житомирська
57885  2024   8.0  0.130  269.80  62.44  36.88  49.66  Житомирська
57886  2024   9.0  0.145  271.81  64.94  35.96  50.45  Житомирська
57887  2024  10.0  0.029  258.26  64.94  35.96  50.45  Житомирська

[57888 rows x 8 columns]


Step 3: Procedures implementation

In [17]:
def VHI_of_year_finder(area, year):
    finded = df[(df["Area"] == area) & (df["Year"] == str(year))]
    for index, row in finded.iterrows():
        print(f"Тиждень: {row['Week']}, VHI: {row['VHI']}")

    min_v = df[(df['Year'].astype(str) == str(year)) & (df["Area"] == area)]['VHI'].min()
    print('Найменший показник VHI у', area, 'за', year, 'рік: ', min_v)
    max_v = df[(df['Year'].astype(str) == str(year)) & (df["Area"] == area)]['VHI'].max()
    print('Найбільший показник VHI у', area, 'за', year, 'рік: ', max_v)

In [18]:
VHI_of_year_finder('Черкаська', 1987)

Тиждень: 1.0, VHI: 44.86
Тиждень: 2.0, VHI: 46.66
Тиждень: 3.0, VHI: 48.18
Тиждень: 4.0, VHI: 46.91
Тиждень: 5.0, VHI: 45.38
Тиждень: 6.0, VHI: 45.14
Тиждень: 7.0, VHI: 43.92
Тиждень: 8.0, VHI: 44.53
Тиждень: 9.0, VHI: 45.65
Тиждень: 10.0, VHI: 47.62
Тиждень: 11.0, VHI: 48.04
Тиждень: 12.0, VHI: 47.25
Тиждень: 13.0, VHI: 46.06
Тиждень: 14.0, VHI: 45.66
Тиждень: 15.0, VHI: 45.84
Тиждень: 16.0, VHI: 45.25
Тиждень: 17.0, VHI: 41.42
Тиждень: 18.0, VHI: 38.13
Тиждень: 19.0, VHI: 38.22
Тиждень: 20.0, VHI: 35.6
Тиждень: 21.0, VHI: 34.61
Тиждень: 22.0, VHI: 36.14
Тиждень: 23.0, VHI: 41.03
Тиждень: 24.0, VHI: 46.38
Тиждень: 25.0, VHI: 51.75
Тиждень: 26.0, VHI: 55.46
Тиждень: 27.0, VHI: 56.73
Тиждень: 28.0, VHI: 57.28
Тиждень: 29.0, VHI: 55.94
Тиждень: 30.0, VHI: 55.47
Тиждень: 31.0, VHI: 57.02
Тиждень: 32.0, VHI: 58.17
Тиждень: 33.0, VHI: 58.66
Тиждень: 34.0, VHI: 58.6
Тиждень: 35.0, VHI: 57.27
Тиждень: 36.0, VHI: 56.16
Тиждень: 37.0, VHI: 56.04
Тиждень: 38.0, VHI: 55.43
Тиждень: 39.0, VHI: 53.

In [19]:
def VHI_of_range_finder(areas, start_year, end_year):
    for year in range(start_year, end_year + 1):
        print(f"Рік: {year}")
        for area in areas:
            filtered_data = df[(df['Year'] == str(year)) & (df['Area'] == area)]
            print(f"Область: {area}")
            for index, row in filtered_data.iterrows():
                print(f"Тиждень: {row['Week']}, VHI: {row['VHI']}")
            print()
        print()

In [20]:
VHI_of_range_finder(['Черкаська', 'Запорізька'], 1987, 1989)

Рік: 1987
Область: Черкаська
Тиждень: 1.0, VHI: 44.86
Тиждень: 2.0, VHI: 46.66
Тиждень: 3.0, VHI: 48.18
Тиждень: 4.0, VHI: 46.91
Тиждень: 5.0, VHI: 45.38
Тиждень: 6.0, VHI: 45.14
Тиждень: 7.0, VHI: 43.92
Тиждень: 8.0, VHI: 44.53
Тиждень: 9.0, VHI: 45.65
Тиждень: 10.0, VHI: 47.62
Тиждень: 11.0, VHI: 48.04
Тиждень: 12.0, VHI: 47.25
Тиждень: 13.0, VHI: 46.06
Тиждень: 14.0, VHI: 45.66
Тиждень: 15.0, VHI: 45.84
Тиждень: 16.0, VHI: 45.25
Тиждень: 17.0, VHI: 41.42
Тиждень: 18.0, VHI: 38.13
Тиждень: 19.0, VHI: 38.22
Тиждень: 20.0, VHI: 35.6
Тиждень: 21.0, VHI: 34.61
Тиждень: 22.0, VHI: 36.14
Тиждень: 23.0, VHI: 41.03
Тиждень: 24.0, VHI: 46.38
Тиждень: 25.0, VHI: 51.75
Тиждень: 26.0, VHI: 55.46
Тиждень: 27.0, VHI: 56.73
Тиждень: 28.0, VHI: 57.28
Тиждень: 29.0, VHI: 55.94
Тиждень: 30.0, VHI: 55.47
Тиждень: 31.0, VHI: 57.02
Тиждень: 32.0, VHI: 58.17
Тиждень: 33.0, VHI: 58.66
Тиждень: 34.0, VHI: 58.6
Тиждень: 35.0, VHI: 57.27
Тиждень: 36.0, VHI: 56.16
Тиждень: 37.0, VHI: 56.04
Тиждень: 38.0, VHI: 

In [22]:
def drought_finder(percentage):
    for year in df['Year'].unique():
        drought_counter = 0
        slight_drought_counter = 0
        for area in df['Area'].unique():
            area_data = df[(df['Year'] == year) & (df['Area'] == area)]
            if (area_data['VHI'] < 15).any():
                drought_counter += 1
            elif (area_data['VHI'] < 35).any():
                slight_drought_counter += 1
        perc_ext = drought_counter/27 * 100
        if perc_ext > percentage:
            print(f'Рік, коли Екстримальні посухи торкнулися більше {percentage}% областей по Україні: ', year, perc_ext)
        perc_norm = slight_drought_counter/27 * 100
        if perc_norm > percentage:
            print(f'Рік, коли Помірні посухи торкнулися більше {percentage}% областей по Україні: ', year, perc_norm)

In [23]:
drought_finder(50)

Рік, коли Помірні посухи торкнулися більше 50% областей по Україні:  1982 100.0
Рік, коли Помірні посухи торкнулися більше 50% областей по Україні:  1983 96.29629629629629
Рік, коли Помірні посухи торкнулися більше 50% областей по Україні:  1984 92.5925925925926
Рік, коли Помірні посухи торкнулися більше 50% областей по Україні:  1985 96.29629629629629
Рік, коли Помірні посухи торкнулися більше 50% областей по Україні:  1986 92.5925925925926
Рік, коли Помірні посухи торкнулися більше 50% областей по Україні:  1987 74.07407407407408
Рік, коли Помірні посухи торкнулися більше 50% областей по Україні:  1988 88.88888888888889
Рік, коли Помірні посухи торкнулися більше 50% областей по Україні:  1989 96.29629629629629
Рік, коли Помірні посухи торкнулися більше 50% областей по Україні:  1990 92.5925925925926
Рік, коли Помірні посухи торкнулися більше 50% областей по Україні:  1991 88.88888888888889
Рік, коли Помірні посухи торкнулися більше 50% областей по Україні:  1992 88.88888888888889
Рік